### *방재기상관측(AWS) 데이터 전처리 일별*
> 250개 시군구 기준으로 날씨를 정리함
> 1. 시군구의 위경도와 센서간의 유클리드 거리로 시군구별 top 4개의 센서를 찾음
> 2. 거리별 가중치를 계산하여 각 날씨 관련 변수를 계산
##### w1 = 1/3 * [1 - {|d-a1|/(|d-a1| + |d-a2| + |d-a3| + |d-a4|)}]
##### w2 = 1/3 * [1 - {|d-a2|/(|d-a1| + |d-a2| + |d-a3| + |d-a4|)}]
##### w3 = 1/3 * [1 - {|d-a3|/(|d-a1| + |d-a2| + |d-a3| + |d-a4|)}]
##### w4 = 1/3 * [1 - {|d-a4|/(|d-a1| + |d-a2| + |d-a3| + |d-a4|)}]
sigungu_xy.csv (시군구)
censerinfo.csv (센서데이터)
OBS_AWS_DD_201701_202010.csv (센서별 날씨관련 변수)

In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import mean,col,split, col, regexp_extract, when, lit

spark = SparkSession.builder.master("local[2]") \
                    .appName('sparkedu') \
                    .getOrCreate()
spark

### 1. 시군구 위경도 로딩하여 전처리

#### 1.1 시군구 위경도 로딩 컬럼 타입 수정 및 sql사용을 위해 데이터셋 등록

In [2]:
SigunguXY = spark\
          .read\
          .option("inferSchema", "true")\
          .option("header", "true")\
          .csv("data/sigungu_xy.csv")
SigunguXY = SigunguXY.withColumn("sigungu_x", SigunguXY.sigungu_x.cast("float")).withColumn("sigungu_y", SigunguXY.sigungu_y.cast("float"))
SigunguXY.createOrReplaceTempView("Sigunguxy")
print(SigunguXY.count())
# 250

250


#### 1.2 검증

In [3]:
print(SigunguXY.count())
print(SigunguXY.distinct().count())
SigunguXY.describe().show()
SigunguXY.show(2)
# 250

250
250
+-------+--------+-------+------------------+------------------+
|summary|    sido|sigungu|         sigungu_x|         sigungu_y|
+-------+--------+-------+------------------+------------------+
|  count|     250|    250|               250|               250|
|   mean|    null|   null|127.63907656860351| 36.41521324157715|
| stddev|    null|   null| 0.912011813226167|1.0729488769413968|
|    min|  강원도| 가평군|         124.67009|         33.254066|
|    max|충청북도| 횡성군|         130.90572|         38.380592|
+-------+--------+-------+------------------+------------------+

+------+-------+---------+---------+
|  sido|sigungu|sigungu_x|sigungu_y|
+------+-------+---------+---------+
|강원도| 강릉시| 128.8759| 37.75211|
|강원도| 고성군|128.46786|38.380592|
+------+-------+---------+---------+
only showing top 2 rows



### 2. 524개의 센서 데이터를 전처리

#### 2.1 데이터 로딩, 컬럼 타입 수정 및 sql사용을 위해 데이터셋 등록

In [4]:
SenserInfo = spark\
          .read\
          .option("inferSchema", "true")\
          .option("header", "true")\
          .csv("data/censerinfo.csv")
SenserInfo = SenserInfo.withColumn("x", SenserInfo.x.cast("float")).withColumn("y", SenserInfo.y.cast("float"))
SenserInfo.createOrReplaceTempView("SenserInfo")
print(SenserInfo.count())
# 1893

1893


#### 2.2 데이터 정리 - 이력관리 되고 있는 센서 위치 정보를 최근으로 데이터로 가져 오고 필요 데이터만 으로 subset을 만듬

In [5]:
print(SenserInfo.distinct().count())
SenserInfo.describe().show()
# SenserInfo.show()
# 1893

1893
+-------+-----------------+-------------------+-------------------+-----------+------------------+------------------+------------------+------------------+------------------+
|summary|         senserid|          startdate|            enddate|sernsername|                 x|                 y|region_1depth_name|region_2depth_name|region_3depth_name|
+-------+-----------------+-------------------+-------------------+-----------+------------------+------------------+------------------+------------------+------------------+
|  count|             1893|               1893|               1893|       1893|              1893|              1893|              1893|              1893|              1893|
|   mean|693.0612783940835|               null|               null|       null|127.53129761419281|36.158902869045704|              null|              null|              null|
| stddev| 174.599815603351|               null|               null|       null|0.9911930175069503|1.3313135706812764|   

In [6]:
LastSenserInfo = spark.sql("""
SELECT A.senserid, A.sernsername, A.x AS senser_x, A.y AS senser_y,
        A.region_1depth_name AS senser_region_1,  
        A.region_3depth_name AS senser_region_2,  
        A.region_2depth_name AS senser_region_3
FROM SenserInfo AS A

INNER JOIN (
    SELECT senserid, max(enddate) AS lenddate
    FROM SenserInfo
    GROUP BY senserid
    ) AS B
ON A.senserid = B.senserid
AND A.enddate = B.lenddate

WHERE 1 = 1

""")
LastSenserInfo.createOrReplaceTempView("LastSenserInfo")
LastSenserInfo.describe().show()
print(LastSenserInfo.count())
# 524

+-------+-----------------+-----------+------------------+------------------+---------------+---------------+---------------+
|summary|         senserid|sernsername|          senser_x|          senser_y|senser_region_1|senser_region_2|senser_region_3|
+-------+-----------------+-----------+------------------+------------------+---------------+---------------+---------------+
|  count|              524|        524|               524|               524|            524|            524|            524|
|   mean|679.4561068702291|       null|127.52589231593008|36.187192130634806|           null|           null|           null|
| stddev|184.2299668080408|       null|1.0150116146748691| 1.364833816970453|           null|           null|           null|
|    min|               12|     가거도|          124.6305|           33.1221|         강원도|         가산면|         가평군|
|    max|              978|       횡성|          131.8698|           38.5439|       충청북도|         흑산면|         횡성군|
+-------+-------

### 3. 시군구 와 날씨 데이터의 위경도 값을 이용하여 유클리드 거리고 top3 선택

#### 3.1 full join

In [7]:
Sigungu_dist = spark.sql("""
SELECT *
FROM SigunguXY AS A

FULL JOIN LastSenserInfo AS B

WHERE 1 = 1
-- AND A.sigungu = '강릉시'

""")
Sigungu_dist.createOrReplaceTempView("Sigungu_dist")
print(Sigungu_dist.count())
Sigungu_dist.show(2)
# 250 * 524 = 131000

131000
+------+-------+---------+---------+--------+-----------+--------+--------+---------------+---------------+---------------+
|  sido|sigungu|sigungu_x|sigungu_y|senserid|sernsername|senser_x|senser_y|senser_region_1|senser_region_2|senser_region_3|
+------+-------+---------+---------+--------+-----------+--------+--------+---------------+---------------+---------------+
|강원도| 강릉시| 128.8759| 37.75211|      12| 안면도(감)|126.3167| 36.5333|       충청남도|         안면읍|         태안군|
|강원도| 고성군|128.46786|38.380592|      12| 안면도(감)|126.3167| 36.5333|       충청남도|         안면읍|         태안군|
+------+-------+---------+---------+--------+-----------+--------+--------+---------------+---------------+---------------+
only showing top 2 rows



#### 3.2 유클리드 거리 및 순위 추출 (pyspark dataframe에서 처리 불가 하여 변환하여처리)

In [8]:
from scipy.spatial import distance
print(distance.euclidean((128.8759061, 37.75210808), (126.3167, 36.5333)))
print(distance.euclidean((128.8759061, 37.75210808), (128.8759061, 37.75210808)))
print(distance.euclidean((0, 0), (365, 365)))

2.8346126716273883
0.0
516.1879502661797


In [9]:
from scipy.spatial import distance
# print(distance.euclidean((128.8759061, 37.75210808), (126.3167, 36.5333)))
print('변환 전', type(Sigungu_dist))
# Sigunguxy_dist['distance'] = Sigunguxy_dist.map(lambda x : distance.euclidean((x['sigungu_x'], x['sigungu_y']), (x['x'], x['y'])), axis=1)

#유클리드 함수가 안 돌아 가서 pdf로 변환 후 처리
pdf_Sigungu_dist = Sigungu_dist.toPandas()
print('변환 후', type(pdf_Sigungu_dist))
pdf_Sigungu_dist['udistance'] = pdf_Sigungu_dist.apply(lambda x : distance.euclidean((x['sigungu_x'], x['sigungu_y']), (x['senser_x'], x['senser_y'])), axis=1)
# 다시 스파크 프레임으로 변환
Sigungu_dist = spark.createDataFrame(pdf_Sigungu_dist)
Sigungu_dist.createOrReplaceTempView("Sigungu_dist")
print('유클리드 거리 추가 후 다시변환', type(Sigungu_dist))
print(Sigungu_dist.count())
Sigungu_dist.show(2)
# 131000

변환 전 <class 'pyspark.sql.dataframe.DataFrame'>
변환 후 <class 'pandas.core.frame.DataFrame'>
유클리드 거리 추가 후 다시변환 <class 'pyspark.sql.dataframe.DataFrame'>
131000
+------+-------+------------------+------------------+--------+-----------+------------------+-----------------+---------------+---------------+---------------+------------------+
|  sido|sigungu|         sigungu_x|         sigungu_y|senserid|sernsername|          senser_x|         senser_y|senser_region_1|senser_region_2|senser_region_3|         udistance|
+------+-------+------------------+------------------+--------+-----------+------------------+-----------------+---------------+---------------+---------------+------------------+
|강원도| 강릉시| 128.8759002685547| 37.75210952758789|      12| 안면도(감)|126.31670379638672|36.53329849243164|       충청남도|         안면읍|         태안군| 2.834605249867365|
|강원도| 고성군|128.46786499023438|38.380592346191406|      12| 안면도(감)|126.31670379638672|36.53329849243164|       충청남도|         안면읍|         태안군|2.8

#### 3.3 유클리드 거리에 따라 작은 top 추출

In [10]:
# 최종 센서데이터에 빠지는 경우가 있기 때문에 전체 여기서 센서의 개수를 정하지 못함 못함 ㅠㅠ
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# 시군구 기준으로 센서들의 udistance에 따라 순위를 줌 
# windowSpec  = Window.partitionBy("sido", "sigungu").orderBy("udistance")
# Sigungu_dist = Sigungu_dist.withColumn("row_number", row_number().over(windowSpec))

# Sigungu_dist.createOrReplaceTempView("Sigungu_dist")
# print(Sigungu_dist.count())
# Sigungu_dist.show(2)
# 131000

In [11]:
# 최종 센서데이터에 빠지는 경우가 있기 때문에 전체 여기서 센서의 개수를 정하지 못함 못함 ㅠㅠ
##### 파일 전달을 위해 그냥 둠 ####
SigunguSenserTop = spark.sql("""
SELECT *
FROM Sigungu_dist 
WHERE 1 = 1
-- AND row_number IN (1, 2, 3, 4)
""")
SigunguSenserTop.createOrReplaceTempView("SigunguSenserTop")
print(SigunguSenserTop.count())
SigunguSenserTop.show(2)
# SigunguSenserTop3.toPandas().to_csv("./SenserTop.csv")
# 250 * 4 = 1000 ->  131000 전체를 다 가져감

131000
+------+-------+------------------+------------------+--------+-----------+------------------+-----------------+---------------+---------------+---------------+------------------+
|  sido|sigungu|         sigungu_x|         sigungu_y|senserid|sernsername|          senser_x|         senser_y|senser_region_1|senser_region_2|senser_region_3|         udistance|
+------+-------+------------------+------------------+--------+-----------+------------------+-----------------+---------------+---------------+---------------+------------------+
|강원도| 강릉시| 128.8759002685547| 37.75210952758789|      12| 안면도(감)|126.31670379638672|36.53329849243164|       충청남도|         안면읍|         태안군| 2.834605249867365|
|강원도| 고성군|128.46786499023438|38.380592346191406|      12| 안면도(감)|126.31670379638672|36.53329849243164|       충청남도|         안면읍|         태안군|2.8354874473456375|
+------+-------+------------------+------------------+--------+-----------+------------------+-----------------+---------------+-----

#### 3.4 센터데이터를 최대한 포함하기 위해 top4로 선택함

In [12]:
# 최종 센서데이터에 빠지는 경우가 있기 때문에 전체 여기서 센서의 개수를 정하지 못함 못함 ㅠㅠ
# maxSql = spark.sql("""
# select count(*)
# from (
#     SELECT senserid, count(*)
#     FROM SigunguSenserTop
#     GROUP BY senserid
# )
# """)
# 524
# 3 : 362
# 4 : 411 ***** 선택
# 5 : 433
# 7 : 458
# maxSql.show()

#### 3.5 거리로 가중치 계산 센서의 가까운 정도에 따라 영향을 많이 주도록 함

In [13]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, sum
# 울산, 남구만 테스트
# SigunguSensertmp = SigunguSenserTop.filter((SigunguSenserTop.sido == '울산광역시') & \
#                         (SigunguSenserTop.sigungu == '남구'))
# SigunguSensertmp.createOrReplaceTempView("SigunguSensertmp")
# windowSp  = Window.partitionBy("sido", "sigungu")
# SigunguSensertmp.withColumn("sumUdist", sum(SigunguSensertmp.udistance).over(windowSp)).show()

# 최종 센서데이터에 빠지는 경우가 있기 때문에 전체 여기서 계산 못함 ㅠㅠ
# windowSp  = Window.partitionBy("sido", "sigungu")
# SigunguSenserTop = SigunguSenserTop.withColumn("sumUdist", sum(SigunguSenserTop.udistance).over(windowSp))

# SigunguSenserTop = SigunguSenserTop.withColumn("weight", (1/3 * (1 - SigunguSenserTop.udistance / SigunguSenserTop.sumUdist)))
# SigunguSenserTop.createOrReplaceTempView("SigunguSenserTop")

# print(SigunguSenserTop.count())
# SigunguSenserTop.show(2)
# SigunguSenserTop.describe().show()

### 4. 일/시간별 센터의 날씨데이터와 시도, 시군구 mapping

#### 4.1 날씨정보 로딩 및 필요 정보 정리

In [14]:
AWS_D_201701_202010 = spark\
          .read\
          .option("inferSchema", "true")\
          .option("charset","euc-kr")\
          .option("header", "true")\
          .csv("data/OBS_AWS_DD_201701_202010.csv")
AWS_D_201701_202010 = AWS_D_201701_202010.toDF("senserid", "sersername", "date", "temp", "ltemp", "ltemptime", "htemp", "htemptime", \
                                               "rainfall", "hwindspeed", "hwindspeedtime", "avgwindspeed", "hwindspeeddeg")
AWS_D_201701_202010.createOrReplaceTempView("AWS_D_201701_202010")
print(AWS_D_201701_202010.count())
AWS_D_201701_202010.show(2)
AWS_D_201701_202010.printSchema()
# 702506

702506
+--------+----------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+
|senserid|sersername|      date|temp|ltemp|ltemptime|htemp|htemptime|rainfall|hwindspeed|hwindspeedtime|avgwindspeed|hwindspeeddeg|
+--------+----------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+
|      12|안면도(감)|2017-01-01| 7.1|  4.7|      527| 10.3|     1324|     0.0|       5.8|          1243|         2.2|        200.2|
|      12|안면도(감)|2017-01-02| 7.4|  5.2|     2322| 11.6|     1314|     0.0|       7.1|          1612|         2.2|          7.5|
+--------+----------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+
only showing top 2 rows

root
 |-- senserid: integer (nullable = true)
 |-- sersername: string (nullable = true)
 |-- date: string (nullable = true)
 |-- temp: double (nullable = true)
 |-- ltemp: double (null

In [40]:
from pyspark.sql.functions import expr
# 최고/최저온도 및 최고풍속 발생 시간은 초단위로 변환하여 가중치 적용 후 시간으로 변환
AWS_D_201701_202010 = AWS_D_201701_202010.withColumn("ltemptime", AWS_D_201701_202010.ltemptime.cast("String")) \
                                .withColumn("htemptime", AWS_D_201701_202010.htemptime.cast("String")) \
                                .withColumn("hwindspeedtime", AWS_D_201701_202010.hwindspeedtime.cast("String"))
AWS_D_201701_202010 = AWS_D_201701_202010.withColumn("ltemptime1", expr("substring(ltemptime, 1, length(ltemptime)-2)").cast("int") * 3600 \
                                                     + expr("substring(ltemptime, length(ltemptime)-1, length(ltemptime))").cast("int") * 60 )

AWS_D_201701_202010.show(2)

# AWS_D_201701_202010.printSchema()

+--------+----------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+----------+
|senserid|sersername|      date|temp|ltemp|ltemptime|htemp|htemptime|rainfall|hwindspeed|hwindspeedtime|avgwindspeed|hwindspeeddeg|ltemptime1|ltemptime2|
+--------+----------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+----------+
|      12|안면도(감)|2017-01-01| 7.1|  4.7|      527| 10.3|     1324|     0.0|       5.8|          1243|         2.2|        200.2|     19620|      1620|
|      12|안면도(감)|2017-01-02| 7.4|  5.2|     2322| 11.6|     1314|     0.0|       7.1|          1612|         2.2|          7.5|     84120|      1320|
+--------+----------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+----------+
only showing top 2 rows



In [16]:
SigunguWeather = spark.sql("""
SELECT A.sido, A.sigungu, B.senserid, B.sersername
       , A.udistance, B.date, B.temp, B.ltemp
       , B.ltemptime
       , B.htemp
       , B.htemptime
       , B.rainfall, B.hwindspeed
       , B.hwindspeedtime
       , B.avgwindspeed, B.hwindspeeddeg
FROM SigunguSenserTop AS A

INNER JOIN AWS_D_201701_202010 AS B
ON A.senserid = B.senserid
AND B.temp is not null
AND B.ltemp is not null 
AND B.htemp is not null
-- 센서 이상으로 온도가 나오지 않는 센서는 제거 하
WHERE 1 = 1

""")
SigunguWeather.createOrReplaceTempView("SigunguWeather")
print(SigunguWeather.count())
SigunguWeather.show(2)
# SigunguWeather.describe().show() # 개 오래 걸림 ㅋㅋ
# 175545250 # 하나의 센서가 여러 sigungu에서 쓰일 수 있음 ㅋㅋㅋㅋ 일 4년치 2억건 
# 온도의 null 제거 후 174405250

174405250
+------+-------+--------+----------+------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+
|  sido|sigungu|senserid|sersername|         udistance|      date|temp|ltemp|ltemptime|htemp|htemptime|rainfall|hwindspeed|hwindspeedtime|avgwindspeed|hwindspeeddeg|
+------+-------+--------+----------+------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+
|강원도| 강릉시|     964|      마량|3.8845102818642188|2019-07-09|22.0| 20.0|      549| 25.5|     1143|     0.0|       7.8|           524|         1.6|         25.3|
|강원도| 강릉시|     964|      마량|3.8845102818642188|2019-07-10|20.8| 20.1|     2208| 22.1|      346|    20.5|      17.1|           534|         3.7|         78.8|
+------+-------+--------+----------+------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+
only showi

#### 4.2 유클리드 거리에 따라 작은 순으로 top4 추출

In [17]:
# 최종 센서데이터에 빠지는 경우가 있기 때문에 전체 여기서 센서의 개수를 정하지 못함 못함 ㅠㅠ
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# 시군구 기준으로 센서들의 udistance에 따라 순위를 줌 
windowSpec  = Window.partitionBy("sido", "sigungu", "date").orderBy("udistance")
SigunguWeather = SigunguWeather.withColumn("row_number", row_number().over(windowSpec))

SigunguWeather.createOrReplaceTempView("SigunguWeather")
print(SigunguWeather.count())
SigunguWeather.show(2)
# 174405250

174405250
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+
|  sido|sigungu|senserid|sersername|          udistance|      date|temp|ltemp|ltemptime|htemp|htemptime|rainfall|hwindspeed|hwindspeedtime|avgwindspeed|hwindspeeddeg|row_number|
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+
|강원도| 강릉시|     524|      강문|0.05938647192316765|2017-04-13|13.7|  6.3|      611| 18.6|     1816|     0.0|      11.5|          2154|         3.8|        225.0|         1|
|강원도| 강릉시|     566|      연곡|0.11451445804311591|2017-04-13|13.6|  3.7|      635| 18.1|     2010|     0.0|      11.7|          2205|         3.2|        241.9|         2|
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+---

In [19]:
# udistance에 따른 순위 중 상위 4개만 추출
SigunguSenserTop4 = spark.sql("""
    SELECT *
    FROM SigunguWeather 
    WHERE 1 = 1
    AND row_number IN (1, 2, 3, 4)
    """)
SigunguSenserTop4.createOrReplaceTempView("SigunguSenserTop4")
print(SigunguSenserTop4.count())
SigunguSenserTop4.show(2)
# 1400000 = 250sigungu * 4senser * 1400days
# 2017	365
# 2018	365
# 2019	365
# 2020	305
# 합게 1400

1400000
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+
|  sido|sigungu|senserid|sersername|          udistance|      date|temp|ltemp|ltemptime|htemp|htemptime|rainfall|hwindspeed|hwindspeedtime|avgwindspeed|hwindspeeddeg|row_number|
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+
|강원도| 강릉시|     524|      강문|0.05938647192316765|2017-04-13|13.7|  6.3|      611| 18.6|     1816|     0.0|      11.5|          2154|         3.8|        225.0|         1|
|강원도| 강릉시|     566|      연곡|0.11451445804311591|2017-04-13|13.6|  3.7|      635| 18.1|     2010|     0.0|      11.7|          2205|         3.2|        241.9|         2|
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+-----

#### 4.3 검증

In [25]:
# 시간 좀 걸림
maxSql = spark.sql("""
    SELECT sido
        , sigungu
        , date
        , count(*)
    FROM SigunguSenserTop4
    WHERE 1 = 1
    GROUP BY sido, sigungu, date
    HAVING count(*) < 4
""")
maxSql.show()

+----+-------+----+--------+
|sido|sigungu|date|count(1)|
+----+-------+----+--------+
+----+-------+----+--------+



In [ ]:
# This function use to print feature with null values and null count 
def null_value_count(df):
  null_columns_counts = []
  numRows = df.count()
  for k in df.columns:
    nullRows = df.where(col(k).isNull()).count()
    if(nullRows > 0):
      temp = k,nullRows
      null_columns_counts.append(temp)
  return(null_columns_counts)

# 겁나 오래 돌아 감
# null_columns_count_list = null_value_count(SigunguSenserTop4)
# null_columns_count_list

In [ ]:
# LastSenserInfo.describe().show()

#### 4.4 센서와 시군구 거리에 의한 가중치 구하기

In [20]:
# w1 = 1/3 * [1 - {|d-a1|/(|d-a1| + |d-a2| + |d-a3| + |d-a4|)}]
# w2 = 1/3 * [1 - {|d-a2|/(|d-a1| + |d-a2| + |d-a3| + |d-a4|)}]
# w3 = 1/3 * [1 - {|d-a3|/(|d-a1| + |d-a2| + |d-a3| + |d-a4|)}]
# w4 = 1/3 * [1 - {|d-a4|/(|d-a1| + |d-a2| + |d-a3| + |d-a4|)}]
from pyspark.sql.functions import row_number, sum

windowSp  = Window.partitionBy("sido", "sigungu", "date")
SigunguSenserTop4 = SigunguSenserTop4.withColumn("sumUdist", sum(SigunguSenserTop4.udistance).over(windowSp))

SigunguSenserTop4 = SigunguSenserTop4.withColumn("weight", (1/3 * (1 - SigunguSenserTop4.udistance / SigunguSenserTop4.sumUdist)))
SigunguSenserTop4.createOrReplaceTempView("SigunguSenserTop4")
print(SigunguSenserTop4.count())
SigunguSenserTop4.show(2)

1400000
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+------------------+------------------+
|  sido|sigungu|senserid|sersername|          udistance|      date|temp|ltemp|ltemptime|htemp|htemptime|rainfall|hwindspeed|hwindspeedtime|avgwindspeed|hwindspeeddeg|row_number|          sumUdist|            weight|
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+------------------+------------------+
|강원도| 강릉시|     524|      강문|0.05938647192316765|2017-04-13|13.7|  6.3|      611| 18.6|     1816|     0.0|      11.5|          2154|         3.8|        225.0|         1|0.5086923993644381|0.2944188721269374|
|강원도| 강릉시|     566|      연곡|0.11451445804311591|2017-04-13|13.6|  3.7|      635| 18.1|     2010|     0.0|      11.7|          2205|     

In [22]:
maxSql = spark.sql("""
    SELECT *
    FROM SigunguSenserTop4
    WHERE 1 = 1
    AND sido = '강원도'
    AND sigungu = '강릉시'
    AND date = '2017-04-13'
""")
maxSql.show()

+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+------------------+-------------------+
|  sido|sigungu|senserid|sersername|          udistance|      date|temp|ltemp|ltemptime|htemp|htemptime|rainfall|hwindspeed|hwindspeedtime|avgwindspeed|hwindspeeddeg|row_number|          sumUdist|             weight|
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+------------------+-------------------+
|강원도| 강릉시|     524|      강문|0.05938647192316765|2017-04-13|13.7|  6.3|      611| 18.6|     1816|     0.0|      11.5|          2154|         3.8|        225.0|         1|0.5086923993644381| 0.2944188721269374|
|강원도| 강릉시|     566|      연곡|0.11451445804311591|2017-04-13|13.6|  3.7|      635| 18.1|     2010|     0.0|      11.7|          2205|         

### 5. 가중치를 적용한 일별 시군구의 날씨 정보 추출

In [23]:
# 각 변수에 가중치를 적용함
SigunguSenserTop4_w = spark.sql("""
    SELECT   *
            ,temp           * weight  AS w_temp
            ,ltemp          * weight  AS w_ltemp
            ,ltemptime      * weight  AS w_ltemptime
            ,htemp          * weight  AS w_htemp
            ,htemptime      * weight  AS w_htemptime
            ,rainfall       * weight  AS w_rainfall
            ,hwindspeed     * weight  AS w_hwindspeed
            ,hwindspeedtime * weight  AS w_hwindspeedtime
            ,avgwindspeed   * weight  AS w_avgwindspeed
            ,hwindspeeddeg  * weight  AS w_hwindspeeddeg
    FROM SigunguSenserTop4 
    WHERE 1 = 1
    """)
SigunguSenserTop4_w.createOrReplaceTempView("SigunguSenserTop4_w")
print(SigunguSenserTop4_w.count())
SigunguSenserTop4_w.show(2)

1400000
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+------------------+------------------+-----------------+------------------+------------------+------------------+-----------------+----------+------------------+-----------------+------------------+-----------------+
|  sido|sigungu|senserid|sersername|          udistance|      date|temp|ltemp|ltemptime|htemp|htemptime|rainfall|hwindspeed|hwindspeedtime|avgwindspeed|hwindspeeddeg|row_number|          sumUdist|            weight|           w_temp|           w_ltemp|       w_ltemptime|           w_htemp|      w_htemptime|w_rainfall|      w_hwindspeed| w_hwindspeedtime|    w_avgwindspeed|  w_hwindspeeddeg|
+------+-------+--------+----------+-------------------+----------+----+-----+---------+-----+---------+--------+----------+--------------+------------+-------------+----------+------------------+--------

In [24]:
# 각 변수에 가중치를 적용함
AWS_D_Weather = spark.sql("""
    SELECT  date  
            ,sido
            ,sigungu
            
            ,sum(w_temp)                 AS w_temp
            ,sum(w_ltemp)                AS w_ltemp
            ,max(w_ltemptime)            AS w_ltemptime
            ,sum(w_htemp)                AS w_htemp
            ,max(w_htemptime)            AS w_htemptime
            ,sum(w_rainfall)             AS w_rainfall
            ,sum(w_hwindspeed)           AS w_hwindspeed
            ,max(w_hwindspeedtime)       AS w_hwindspeedtime
            ,sum(w_avgwindspeed)         AS w_avgwindspeed
            ,max(w_hwindspeeddeg)        AS w_hwindspeeddeg
            
    FROM SigunguSenserTop4_w 
    WHERE 1 = 1
    GROUP BY date  
            ,sido
            ,sigungu
    """)
AWS_D_Weather.createOrReplaceTempView("AWS_D_Weather")
print(AWS_D_Weather.count())
AWS_D_Weather.show(2)
# 250 * 1400 = 350000

350000
+----------+------+-------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+
|      date|  sido|sigungu|            w_temp|           w_ltemp|      w_ltemptime|           w_htemp|       w_htemptime|       w_rainfall|     w_hwindspeed|  w_hwindspeedtime|    w_avgwindspeed|   w_hwindspeeddeg|
+----------+------+-------+------------------+------------------+-----------------+------------------+------------------+-----------------+-----------------+------------------+------------------+------------------+
|2017-04-13|강원도| 강릉시|12.338542378567492| 2.609247902321742|615.8865467309814|17.901111684137774|1832.3496860954856|              0.0|11.41930447438681|2156.5406214167415|2.8934348901687326|229.57414959946763|
|2017-07-10|강원도| 강릉시| 24.28263807177009|22.197252267083215|717.4581367144532| 27.07465688441703|1537.2164144231956|9.243996505561913| 6.713

In [28]:
pdf_AWS_D_Weather = AWS_D_Weather.toPandas()
pdf_AWS_D_Weather.to_csv("./AWS_D_Weather.csv", index = False)

In [30]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

# matplotlib 한글 폰트 오류 문제 해결
from matplotlib import font_manager, rc
font_path = "C:/PyStexam/data/THEdog.ttf"   #폰트파일의 위치
font_name = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font_name)

plt.figure(figsize=(35, 15))
sns.lineplot(x="date", y="w_temp", hue="sigungu", data=pdf_AWS_D_Weather)
plt.show()

ValueError: Could not interpret value `sigu` for parameter `hue`

<Figure size 2520x1080 with 0 Axes>

In [ ]:
sns.boxplot(x='date', y='w_temp', hue='sigungu', data=pdf_AWS_D_Weather) 
plt.show()

KeyboardInterrupt: 

In [ ]:
plt.show()